In [226]:
import pandas as pd
import numpy as np
import calendar
import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go

In [214]:
def monthdelta(date, delta):
    '''
    delta could be 1,3,6,12.
    '''
    year = int(date.strftime("%Y"))
    month = int(date.strftime("%m"))
    if month <= delta:
        new_month = month-delta+12
        day = calendar.monthrange(year-1, new_month)[1]
        report_date_prev = datetime.datetime(year - 1, new_month, day)
    else:
        day = calendar.monthrange(year, month-delta)[1]
        report_date_prev = datetime.datetime(year, month - delta, day)
        
    return report_date_prev

In [225]:
def prep_investment_perf_table(data, report_date):
    data['Date'] = pd.to_datetime(data['Date'])
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y')
    last_index = data[data['Date']==date_time_obj].last_valid_index()
    data_cut = data.loc[:last_index,:]
    data_cut = data_cut.loc[((data_cut['Rollup1']=='Investment') | (data_cut['Rollup1']=='Benchmark')),: ]

    data_pivot = pd.pivot_table(data_cut, values = 'Amount', index = ['Date'],
                              columns=['Rollup1'],aggfunc=np.sum)
    data_pivot = data_pivot.reset_index()

    report_date_timeobj = datetime.datetime.strptime(report_date, '%m/%d/%Y')
    report_year = report_date_timeobj.year

    inv_curr = data_pivot.loc[data_pivot['Date']==report_date_timeobj,['Investment']].values[0][0]
    sp_curr = data_pivot.loc[data_pivot['Date']==report_date_timeobj,['Benchmark']].values[0][0]

    year_end = datetime.datetime(report_year-1, 12, 31)
    inv_year_end = data_pivot.loc[data_pivot['Date']==year_end,['Investment']].values[0][0]
    sp_year_end = data_pivot.loc[data_pivot['Date']==year_end,['Benchmark']].values[0][0]

    M3_end = monthdelta(report_date_timeobj, 3)
    inv_M3_end = data_pivot.loc[data_pivot['Date']==M3_end,['Investment']].values[0][0]
    sp_M3_end = data_pivot.loc[data_pivot['Date']==M3_end,['Benchmark']].values[0][0]

    M6_end = monthdelta(report_date_timeobj, 6)
    inv_M6_end = data_pivot.loc[data_pivot['Date']==M6_end,['Investment']].values[0][0]
    sp_M6_end = data_pivot.loc[data_pivot['Date']==M6_end,['Benchmark']].values[0][0]

    M12_end = monthdelta(report_date_timeobj, 12)
    inv_M12_end = data_pivot.loc[data_pivot['Date']==M12_end,['Investment']].values[0][0]
    sp_M12_end = data_pivot.loc[data_pivot['Date']==M12_end,['Benchmark']].values[0][0]

    YTD_return_inv = (inv_curr - inv_year_end)/inv_year_end
    M3_return_inv = (inv_curr - inv_M3_end)/inv_M3_end
    M6_return_inv = (inv_curr - inv_M6_end)/inv_M6_end
    M12_return_inv = (inv_curr - inv_M12_end)/inv_M12_end
    YTD_return_sp = (sp_curr - sp_year_end)/sp_year_end
    M3_return_sp = (sp_curr - sp_M3_end)/sp_M3_end
    M6_return_sp = (sp_curr - sp_M6_end)/sp_M6_end
    M12_return_sp = (sp_curr - sp_M12_end)/sp_M12_end

    investment_perf_df = pd.DataFrame(data={'':['Investment','S&P 500 Index'],
                                            'YTD':[YTD_return_inv,YTD_return_sp],
                                            '3-month':[M3_return_inv,M3_return_sp],
                                            '6-month':[M6_return_inv,M6_return_sp],
                                            '1-year':[M12_return_inv,M12_return_sp]})
    return investment_perf_df

In [228]:
def prep_asset_detail(data, report_date):
    name_list, y_curr_list, y_prev_list = prep_top_mover_data(data, report_date)
    change_list = [a_i - b_i for a_i, b_i in zip(y_curr_list, y_prev_list)]
    df = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data={'Name': name_list, 'Amount': y_curr_list, 'Change': change_list})
    df.drop(df[df['Amount'] < 10].index, inplace = True)
    return df

In [94]:
def prep_tier_asset_data(data, report_date):
    data['Date'] = pd.to_datetime(data['Date'])
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y')
    last_index = data[data['Date']==date_time_obj].last_valid_index()

    data_cut = data.loc[:last_index,:]
    date_list = []
    tier1_list = []
    tier2_list = []
    tier3_list = []
    for d in np.unique(data_cut['Date']):
        tier1_amount = data.loc[(data['Date']==d) & (data['Rollup1']=='Deposit'),'Amount'].sum()
        tier2_amount = data.loc[(data['Date']==d) & ((data['Rollup1']=='Investment')|(data['Rollup1']=='ESP')),'Amount'].sum()
        tier3_amount = data.loc[(data['Date']==d) & (data['Rollup3']=='Asset'),"Amount"].sum() - tier1_amount - tier2_amount
        date_list.append(d)
        tier1_list.append(tier1_amount)
        tier2_list.append(tier2_amount)
        tier3_list.append(tier3_amount)
    
    date_list = pd.to_datetime(date_list)
    date_list = [date_obj.strftime('%m-%d-%Y') for date_obj in date_list]
        
    return date_list, tier1_list, tier2_list, tier3_list

In [106]:
def tier_stack_area(data, report_date):
    date_list, tier1_list, tier2_list, tier3_list = prep_tier_asset_data(data, report_date)
    y_range = [max(tier3_list)*0.8, (max(tier1_list)+max(tier2_list)+max(tier3_list))*1.05]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=date_list, y=tier3_list,
        name='Tier 3',
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(131, 90, 241)'),
        stackgroup='one' # define stack group
    ))
    fig.add_trace(go.Scatter(
        x=date_list, y=tier2_list,
        name='Tier 2',
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(111, 231, 219)'),
        stackgroup='one'
    ))
    fig.add_trace(go.Scatter(
        x=date_list, y=tier1_list,
        name='Tier 1',
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(184, 247, 212)'),
        stackgroup='one'
    ))

    fig.update_layout(yaxis_range=y_range)
    return fig

In [108]:
def prep_top_mover_data(data, report_date):
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y').date()

    year = int(date_time_obj.strftime("%Y"))
    month = int(date_time_obj.strftime("%m"))
    if month == 1:
        day = calendar.monthrange(year, month)[1]
        report_date_prev = datetime.date(year - 1, 12, day)
    else:
        day = calendar.monthrange(year, month-1)[1]
        report_date_prev = datetime.date(year, month -1, day)

    if month <= 10 and month >1:
        report_date_prev = report_date_prev.strftime('%m/%d/%Y')[1:]
    else: 
        report_date_prev = report_date_prev.strftime('%m/%d/%Y')

    data_prep = data.loc[((data['Date'] == report_date)|(data['Date'] == report_date_prev)),['Date','Item','Amount']]
    data_prep = data_prep.loc[data['Rollup3']=='Asset',:]
    data_curr = data_prep.loc[data['Date']==report_date,:]
    data_prev = data_prep.loc[data['Date']==report_date_prev,:]

    df_result = pd.merge(data_curr, data_prev, on='Item')
    df_result['Delta'] = abs(df_result['Amount_x'] - df_result['Amount_y'])
    df_result = df_result.sort_values(by=['Delta'],ascending=False)

    x_list = df_result['Item'].tolist()
    y_curr_list = df_result['Amount_x'].tolist()
    y_prev_list = df_result['Amount_y'].tolist()
    return x_list, y_curr_list, y_prev_list


In [227]:
def get_hist_asset_data(data, report_date):
    data['Date'] = pd.to_datetime(data['Date'])
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y')
    last_index = data[data['Date']==date_time_obj].last_valid_index()

    data_cut = data.loc[:last_index,:]

    data_cut = data_cut.loc[((data_cut['Rollup1'] == 'Deposit')| (data_cut['Rollup1'] == 'Investment')),['Date','Amount']]

    data_pivot = pd.pivot_table(data_cut, values = 'Amount', index = ['Date'],
                          aggfunc=np.sum)
    data_pivot = data_pivot.reset_index()
    data_final = data_pivot.sort_values(by='Date', ascending=False)
    data_final['Date'] = data_final['Date'].dt.strftime('%m/%d/%Y')
    
    if data_final.shape[0] > 6:
        data_final = data_final[:6]
    return data_final


In [59]:
def finance_metric_calc(data,report_date):
    assets = data.loc[((data['Date']==report_date) & (data['Rollup3'] == 'Asset')),'Amount'].sum()
    liabilities = data.loc[((data['Date']==report_date) & (data['Rollup3'] == 'Liability')),'Amount'].sum()
    net_worth = assets - liabilities    
    return net_worth, assets, liabilities

In [106]:
def waterfall_data_prep(data, report_date):
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y').date()

    year = int(date_time_obj.strftime("%Y"))
    month = int(date_time_obj.strftime("%m"))
    if month == 1:
        day = calendar.monthrange(year, month)[1]
        report_date_prev = datetime.date(year - 1, 12, day)
    else:
        day = calendar.monthrange(year, month-1)[1]
        report_date_prev = datetime.date(year, month -1, day)

    if month <= 10 and month >1:
        report_date_prev = report_date_prev.strftime('%m/%d/%Y')[1:]
    else: 
        report_date_prev = report_date_prev.strftime('%m/%d/%Y')

    x_list = [report_date_prev, "Deposit", 'Investment', "ESP", "401k", 'Real Estate', 'Mortgage','Loan',report_date]

    net_worth_prev = finance_metric_calc(data,report_date_prev)[0]
    net_worth_curr = finance_metric_calc(data,report_date)[0]
    y_list = [net_worth_prev]

    for i in x_list[1:-1]:
        delta_i = data.loc[((data['Date']==report_date) & (data['Rollup1'] == i)),'Amount'].sum() - data.loc[((data['Date']==report_date_prev) & (data['Rollup1'] == i)),'Amount'].sum()
        if i in ['Mortgage','Loan']:
            y_list.append(-delta_i)
        else:
            y_list.append(delta_i)

    y_list.append(net_worth_curr)
    y_range = [y_list[0]*0.9,y_list[-1]*1.1]
    return x_list, y_list, y_range

In [147]:

def line_chart_data_prep(data, report_date, account):
    data['Date'] = pd.to_datetime(data['Date'])
    date_time_obj = datetime.datetime.strptime(report_date, '%m/%d/%Y')
    last_index = data[data['Date']==date_time_obj].last_valid_index()
    
    data_cut = data.loc[:last_index,:]
    data_sp = data_cut.loc[data_cut['Rollup3']=='SP500',:]
    if account == 'Asset':
        data_asset = data_cut.loc[data_cut['Rollup3']=='Asset',:]
        data_pivot = pd.pivot_table(data_asset, values = 'Amount', index = ['Date'],
                      columns=['Rollup3'],aggfunc=np.sum)
        y_list = data_pivot[account].tolist()
    else :
        data_asset = data_cut.loc[data_cut['Rollup1']==account,:]
        data_pivot = pd.pivot_table(data_asset, values = 'Amount', index = ['Date'],
                      columns=['Rollup1'],aggfunc=np.sum)
        y_list = data_pivot[account].tolist()
        
    y_list_sp = data_sp['Amount'].tolist()
    date_list = data_pivot.index.tolist()
    
    return date_list, y_list, y_list_sp

In [262]:
def pie_chart_data_prep(data,report_date):
    data_cut = data.loc[(data['Date']==report_date) & (data['Rollup3']=='Asset'),:]
    data_pivot = pd.pivot_table(data_cut, values = 'Amount', index = ['Date'],
                               columns=['Rollup1'], aggfunc=np.sum)
    labels_list = data_pivot.columns.to_list()
    values_list = data_pivot.iloc[0].values.tolist()
    return labels_list, values_list

In [168]:
tier1_amount = data.loc[(data['Date']==report_date) & (data['Rollup1']=='Deposit'),'Amount'].sum()
tier2_amount = data.loc[(data['Date']==report_date) & ((data['Rollup1']=='Investment')|(data['Rollup1']=='ESP')),'Amount'].sum()
tier3_amount = data.loc[(data['Date']==report_date) & (data['Rollup3']=='Asset'),"Amount"].sum() - tier1_amount - tier2_amount

In [146]:
data = pd.read_csv('Records.csv')
report_date = '2/29/2016'
data.head(24)

,Date,Item,Rollup1,Rollup2,Rollup3,Amount
0,1/31/2015,PNC Bank,Deposit,NaN,Asset,5481.550000
1,1/31/2015,Wells Fargo Bank,Deposit,NaN,Asset,0.000000
2,1/31/2015,Capital One Bank,Deposit,NaN,Asset,10145.600000
3,1/31/2015,Robinhood,Investment,NaN,Asset,19770.290000
4,1/31/2015,Scottrade,Investment,NaN,Asset,6347.400000
5,1/31/2015,Lending Club,Investment,NaN,Asset,5442.250000
6,1/31/2015,Google,ESP,NaN,Asset,11137.421880
7,1/31/2015,Merill Lynch,401k,NaN,Asset,26027.930000
8,1/31/2015,House 1,Real Estate,NaN,Asset,570000.000000
9,1/31/2015,Mortgage,Mortgage,NaN,Liability,434171.250000


In [261]:
def prep_liquid_table(data, report_date):
    data_cut = data.loc[((data['Date']==report_date)&(data['Rollup3']=='Asset')),: ]

    data_pivot = pd.pivot_table(data_cut, values = 'Amount', index = ['Date'],
                              columns=['Rollup1'],aggfunc=np.sum)
    data_pivot = data_pivot.reset_index()

    total_asset = 0
    for i in data_pivot.columns.tolist()[1:]:
        total_asset += data_pivot[i][0]

    high_ratio=(data_pivot['Deposit'][0]+data_pivot['Investment'][0])/total_asset
    med_ratio=(data_pivot['ESP'][0]+data_pivot['401k'][0])/total_asset
    low_ratio=data_pivot['Real Estate'][0]/total_asset



    liquidity_df = pd.DataFrame(data={'Account':['Deposit','Investment','ESP','401k','Real Estate','','','',''],
                                     'Liquidity':['High','High','Medium','Medium','Low','Ratio','High','Medium','Low'],
                                     'Amount':[data_pivot['Deposit'][0],data_pivot['Investment'][0],data_pivot['ESP'][0],data_pivot['401k'][0],data_pivot['Real Estate'][0],'',high_ratio,med_ratio,low_ratio]})
    return liquidity_df

Rollup1,Date,401k,Deposit,ESP,Investment,Real Estate
0,2016-02-29,38142.50516,47850.95,33522.75054,37508.328845,581400.0


,Account,Liquidity,Amount
0,Deposit,High,47850.9
1,Investment,High,37508.3
2,ESP,Medium,33522.8
3,401k,Medium,38142.5
4,Real Estate,Low,581400
5,,Ratio,
6,,High,0.115596
7,,Medium,0.0970516
8,,Low,0.787352


In [266]:
data_cut = data.loc[data['Date']==report_date,:]
data_pivot = pd.pivot_table(data_cut, values = 'Amount', index = ['Date'],
                           columns=['Rollup1'], aggfunc=np.sum)
deposit=data_pivot['Deposit'].values.tolist()[0]
investment=data_pivot['Investment'].values.tolist()[0]
esp=data_pivot['ESP'].values.tolist()[0]
i401k=data_pivot['401k'].values.tolist()[0]
re_amount=data_pivot['Real Estate'].values.tolist()[0]
mortgage=data_pivot['Mortgage'].values.tolist()[0]
loan=data_pivot['Loan'].values.tolist()[0]

In [276]:
data_pivot

Rollup1,401k,Benchmark,Deposit,ESP,Investment,Loan,Mortgage,Real Estate
Date,,,,,,,,
2016-02-29,38142.50516,2021.95,47850.95,33522.75054,37508.328845,6510.6875,392030.97,581400.0
